# Preprocessing - augmentation du fichier de résultat "brut" avec données statistiques

Ajout au fichier de résultats bruts de statistiques communales (en se basant sur num OFS) :
- population suisse et étrangère
- éventuellement autre chose (TODO)

In [59]:
# augmenter les résultats en ajoutant des données statistiques (population) issus de la statistique cantonale
# à appliquer après "get-results" et avant "simul-election"

import pandas as pd
import requests

In [5]:
results = pd.read_excel('2017-resultats-communes.xlsx')
results.head()

,1. CHRISTLICHSOZIALE PARTEI (CSP),1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP),1. LES VERTS,1. LES VERTS - GRÜNE,1. PARTI DÉMOCRATE-CHRÉTIEN,1. PARTI LIBÉRAL-RADICAL,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS,2. LES VERTS,"2. SP, GRÜNE UND UNABHÄNGIGE",...,Sièges,Moyenne suffrages/bulletin,Electorat PS,Electorat PDC,Electorat PLR,Electorat CVP,Electorat RCV,Electorat CSP,Electorat UDC,Electorat Verts
Bellwald,0,121,0,0,0,0,0,0,0,38,...,2,1.983740,19.155738,0.0,0.0,29.741803,0.0,60.995902,13.106557,0.0
Binn,0,49,0,0,0,0,0,0,0,8,...,2,1.980392,4.039604,0.0,0.0,17.168317,0.0,24.742574,5.049505,0.0
Ernen,0,121,0,0,0,0,0,0,0,61,...,2,1.990385,30.647343,0.0,0.0,85.913043,0.0,60.792271,30.647343,0.0
Fiesch,0,325,0,0,0,0,0,0,0,63,...,2,1.984802,31.741194,0.0,0.0,71.543645,0.0,163.744257,61.970904,0.0
Fieschertal,0,104,0,0,0,0,0,0,0,13,...,2,1.992000,6.526104,0.0,0.0,21.084337,0.0,52.208835,45.180723,0.0


In [4]:
stats = pd.read_excel('population-vs-2018.xlsx')
stats.head()

,Nr,Commune,Population,Population-hommes,Population-femmes,Suisses,Suisses-hommes,Suisses-femmes,Etrangers,Etrangers-hommes,Etrangers-femmes
0,6052,Bellwald,378,204,174,323,175,148,55,29,26
1,6054,Binn,142,76,66,135,73,62,7,3,4
2,6056,Ernen,492,240,252,439,215,224,53,25,28
3,6057,Fiesch,921,491,430,803,426,377,118,65,53
4,6058,Fieschertal,318,159,159,285,141,144,33,18,15


In [9]:
communes_stats = list(stats['Commune'])
communes_results = list(results.index)

In [10]:
# missing
missing_in_stats = list(set(communes_results) - set(communes_stats))
missing_in_stats

['St-Maurice',
 'St-Martin',
 'St-Gingolph',
 'St-Léonard',
 'Wiler',
 'Bourg-St-Pierre']

In [11]:
# missing
missing_in_results = list(set(communes_stats) - set(communes_results))
missing_in_results

['Bourg-Saint-Pierre',
 'Saint-Gingolph',
 'Saint-Léonard',
 'Wiler (Lötschen)',
 'Saint-Martin',
 'Saint-Maurice']

In [15]:
# fix data -> add column with OFS correct names

def fixOFSCommune(commune):
    fixes = {
         'St-Maurice':'Saint-Maurice',
         'St-Gingolph':'Saint-Gingolph',
         'Bourg-St-Pierre':'Bourg-Saint-Pierre',
         'St-Léonard':'Saint-Léonard',
         'Wiler':'Wiler (Lötschen)',
         'St-Martin':'Saint-Martin'
    } 
    if commune in fixes:
        return fixes[commune]
    else:
        return commune

results['Commune'] = results.apply(lambda x: fixOFSCommune(x.name), axis=1)

In [23]:
# add OFS number while we're here

maps = { item['Commune'] : item['Nr'] for item in stats[['Commune','Nr']].to_dict("records") }
results['OFS'] = results['Commune'].apply(lambda x: maps[x])
results.head()

,1. CHRISTLICHSOZIALE PARTEI (CSP),1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP),1. LES VERTS,1. LES VERTS - GRÜNE,1. PARTI DÉMOCRATE-CHRÉTIEN,1. PARTI LIBÉRAL-RADICAL,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS,2. LES VERTS,"2. SP, GRÜNE UND UNABHÄNGIGE",...,Electorat PS,Electorat PDC,Electorat PLR,Electorat CVP,Electorat RCV,Electorat CSP,Electorat UDC,Electorat Verts,Commune,OFS
Bellwald,0,121,0,0,0,0,0,0,0,38,...,19.155738,0.0,0.0,29.741803,0.0,60.995902,13.106557,0.0,Bellwald,6052
Binn,0,49,0,0,0,0,0,0,0,8,...,4.039604,0.0,0.0,17.168317,0.0,24.742574,5.049505,0.0,Binn,6054
Ernen,0,121,0,0,0,0,0,0,0,61,...,30.647343,0.0,0.0,85.913043,0.0,60.792271,30.647343,0.0,Ernen,6056
Fiesch,0,325,0,0,0,0,0,0,0,63,...,31.741194,0.0,0.0,71.543645,0.0,163.744257,61.970904,0.0,Fiesch,6057
Fieschertal,0,104,0,0,0,0,0,0,0,13,...,6.526104,0.0,0.0,21.084337,0.0,52.208835,45.180723,0.0,Fieschertal,6058


In [56]:
# merge avec fichier population (totale-suisse-étrangers)
# remettre le nom de commune comme index (même si c'est pas le nom OFS)
merge = results.reset_index().merge(stats,left_on='OFS',right_on='Nr').set_index('index')
merge.index.name = None
merge['Commune'] = merge['Commune_x']
merge = merge.drop(['Commune_x','Commune_y'],axis=1) # supprimer le champ à double 
merge.columns

Index(['1. CHRISTLICHSOZIALE PARTEI (CSP)',
       '1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP)', '1. LES VERTS',
       '1. LES VERTS - GRÜNE', '1. PARTI DÉMOCRATE-CHRÉTIEN',
       '1. PARTI LIBÉRAL-RADICAL',
       '2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS',
       '2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS', '2. LES VERTS',
       '2. SP, GRÜNE UND UNABHÄNGIGE', '2. UDC',
       '3. ALLIANCE DE GAUCHE (PARTI SOCIALISTE, CENTRE GAUCHE-PCS)',
       '3. ALLIANCE DE GAUCHE - ENTREMONT AUTREMENT',
       '3. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS',
       '3. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS', '3. PLR / FDP',
       '3. PLR.LES LIBÉRAUX-RADICAUX - FDP.DIE LIBERALEN',
       '3. SCHWEIZERISCHE VOLKSPARTEI OBERWALLIS (SVPO)',
       '4. PARTI DÉMOCRATE-CHRÉTIEN',
       '4. PARTI DÉMOCRATE-CHRÉTIEN (PDC) - CHRISTLICHDEMOKRATISCHE VOLKSPARTEI',
       '4. PARTI DÉMOCRATE-CHRÉTIEN - CHRISTLICHDEMOKRATISCHE VOLKSPARTEI',
       '4. PARTI

In [58]:
with pd.ExcelWriter('2017-resultats-communes-augmente.xlsx') as writer:
    merge.to_excel(writer)

In [57]:
merge.head()

,1. CHRISTLICHSOZIALE PARTEI (CSP),1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP),1. LES VERTS,1. LES VERTS - GRÜNE,1. PARTI DÉMOCRATE-CHRÉTIEN,1. PARTI LIBÉRAL-RADICAL,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS,2. LES VERTS,"2. SP, GRÜNE UND UNABHÄNGIGE",...,Population,Population-hommes,Population-femmes,Suisses,Suisses-hommes,Suisses-femmes,Etrangers,Etrangers-hommes,Etrangers-femmes,Commune
Bellwald,0,121,0,0,0,0,0,0,0,38,...,378,204,174,323,175,148,55,29,26,Bellwald
Binn,0,49,0,0,0,0,0,0,0,8,...,142,76,66,135,73,62,7,3,4,Binn
Ernen,0,121,0,0,0,0,0,0,0,61,...,492,240,252,439,215,224,53,25,28,Ernen
Fiesch,0,325,0,0,0,0,0,0,0,63,...,921,491,430,803,426,377,118,65,53,Fiesch
Fieschertal,0,104,0,0,0,0,0,0,0,13,...,318,159,159,285,141,144,33,18,15,Fieschertal


In [45]:
merge.index

Index(['Bellwald', 'Binn', 'Ernen', 'Fiesch', 'Fieschertal', 'Goms', 'Lax',
       'Obergoms', 'Bettmeralp', 'Bister',
       ...
       'Vérossaz', 'Champéry', 'Collombey-Muraz', 'Monthey', 'Port-Valais',
       'Saint-Gingolph', 'Troistorrents', 'Val-d'Illiez', 'Vionnaz', 'Vouvry'],
      dtype='object', name='', length=126)

In [47]:
results.index

Index(['Bellwald', 'Binn', 'Ernen', 'Fiesch', 'Fieschertal', 'Goms', 'Lax',
       'Obergoms', 'Bettmeralp', 'Bister',
       ...
       'Vérossaz', 'Champéry', 'Collombey-Muraz', 'Monthey', 'Port-Valais',
       'St-Gingolph', 'Troistorrents', 'Val-d'Illiez', 'Vionnaz', 'Vouvry'],
      dtype='object', length=126)